In [2]:
import sqlite3
import json
from datetime import datetime



In [3]:
######################
#TUTORIAL code-block 1
######################

timeframe = '2009-04'
#build one transaction and connect data all at once
sql_transaction = []

connection = sqlite3.connect('{}.db'.format(timeframe))
#cursor
c = connection.cursor()

def create_table() :
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply
    (parent_id TEXT PRIMARY KEY, comment_id TEXT UNIQUE, 
    parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)""")

def format_data(data):
    data = data.replace("\n", " newlinechar ").replace("\r", " newlinechar ").replace('"', "'")
    return data

def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True
    


def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0] # 0 cause we are only selecting 1 commenting
        else:
            return False
    except Exception as e:
        #print("Find Parent)
        return False
    

In [4]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0] # 0 cause we are only selecting 1 commenting
        else:
            return False
    except Exception as e:
        #print("Find Parent)
        return False 

In [13]:
def transaction_bldr(sql):
    #globalling the sql_trnsaction variable
    global sql_transaction
    #keep building it till it's a certain size
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
    connection.commit()
    sql_transaction = []


def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?,
        parent = ?, comment = ?, subreddit = ?, unix = ?, score = ?
        WHERE parent_id = ?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s0 insertion', e.unicode)
        
    
def sql_insert_has_parent(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment,
        subreddit, unix, score) VALUES ("{}","{}","{}","{}","{}",{},
        {});""".format(parentid, commentid, parent, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s1 insertion',e.unicode)


def sql_insert_no_parent(commentid,parentid,comment,subreddit,time,score):
    try:
        unicode(comment, "ascii")
    except UnicodeError:
        comment = unicode(comment, "utf-8")
    else:
        # value was valid ASCII data
        pass
    
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, comment, subreddit,
        unix, score) VALUES ("{}","{}","{}","{}",{},
        {});""".format(parentid, commentid, comment, subreddit, int(time), score)
        transaction_bldr(sql)
    except Exception as e:
        print('s2 insertion',e.unicode)   
        
        


In [14]:
#parent_data = find_parent(parent_id)    
if __name__ == "__main__":
        create_table()
        #how many rows we have created
        row_counter = 0
        #how many parents and child pairs we found
        paired_rows = 0
        
        with open('RC_2009-04', buffering = 1000) as f:
            for row in f:
                row_counter += 1
                row = json.loads(row)
                parent_id = row['parent_id']
                body = format_data(row['body'])
                comment_id = row['name']
                created_utc = row['created_utc']
                score = row['score']
                subreddit = row['subreddit']
                parent_data = find_parent(parent_id)
                
                
                
                #since there is a lot of useless comments on reddit,
                #we only keep certain comments
                if score >= 2: #means somebody saw this comment at least 
                    if acceptable(body):
                        #now we check to see if there's a comment with this parent id with a better score
                        existing_comment_score = find_existing_score(parent_id)
                        if existing_comment_score:
                            if score > existing_comment_score:
                                ####
                                sql_insert_replace_comment(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                        else:
                            if parent_data:
                                sql_insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score)
                            else:
                                sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)
                            
                    
                    

TypeError: decoding Unicode is not supported

In [ ]:
######################
#TUTORIAL code-block 1
######################


